# Notebook 05 — ML-Based Signal Diagnostics (Reframed)

**Adaptive Pair Trading using Cointegration and Volatility Modeling**  
**Author:** Ayush Arora (MQMS2404)

---

## Objective

This notebook uses machine learning models as **probabilistic diagnostic tools** to examine whether
short-horizon mean-reversion events in a stationary spread exhibit learnable predictive structure.

Given the rarity and stochastic nature of mean-reversion events, model outputs are interpreted as
risk-aware signals rather than deterministic classifiers.

## Cell 1: Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

## Cell 2: Load spread, Z-score, and volatility

In [3]:
spread = pd.read_csv('data\spread_tatasteel_hindalco.csv', index_col=0, parse_dates=True).iloc[:,0]
zscore = pd.read_csv('data\zscore_tatasteel_hindalco.csv', index_col=0, parse_dates=True).iloc[:,0]
garch_vol = pd.read_csv('data\garch_vol_tatsteel_hindalco.csv', index_col=0, parse_dates=True).iloc[:,0]

spread.head(), zscore.head(), garch_vol.head()

(Date
 2015-01-01   -7.063013
 2015-01-02   -6.987348
 2015-01-05   -5.824623
 2015-01-06   -6.484915
 2015-01-07   -6.006633
 Name: 0, dtype: float64,
 Date
 2015-01-01   -0.312319
 2015-01-02   -0.302455
 2015-01-05   -0.150890
 2015-01-06   -0.236962
 2015-01-07   -0.174616
 Name: 0, dtype: float64,
 Date
 2015-01-02    0.643896
 2015-01-05    0.631061
 2015-01-06    0.669407
 2015-01-07    0.671586
 2015-01-08    0.666134
 Name: garch_vol, dtype: float64)

## Cell 3: Feature construction

In [4]:
df = pd.DataFrame({
    'spread': spread,
    'zscore': zscore,
    'spread_lag1': spread.shift(1),
    'spread_lag2': spread.shift(2),
    'volatility': garch_vol
})

df = df.dropna()
df.head()

,spread,zscore,spread_lag1,spread_lag2,volatility
Date,,,,,
2015-01-05,-5.824623,-0.150890,-6.987348,-7.063013,0.631061
2015-01-06,-6.484915,-0.236962,-5.824623,-6.987348,0.669407
2015-01-07,-6.006633,-0.174616,-6.484915,-5.824623,0.671586
2015-01-08,-6.402756,-0.226252,-6.006633,-6.484915,0.666134
2015-01-09,-6.622972,-0.254958,-6.402756,-6.006633,0.658258


## Cell 4: Mean-reversion label

In [5]:
horizon = 5
future_spread = df['spread'].shift(-horizon)

df['revert'] = (
    (df['spread'] * future_spread < 0) |
    (np.abs(future_spread) < 0.8 * np.abs(df['spread']))
).astype(int)

df = df.dropna()
df['revert'].value_counts(normalize=True)

revert
0    0.715213
1    0.284787
Name: proportion, dtype: float64

## Cell 5: Train-test split (no shuffling)

In [6]:
X = df.drop(columns='revert')
y = df['revert']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=False
)

## Cell 6: Logistic Regression (probability diagnostics)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)

probs_logit = logit.predict_proba(X_test_scaled)[:,1]

for thr in np.arange(0, 1, 0.1):
    preds = (probs_logit > thr).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, preds, average='binary', zero_division=0
    )
    print(f"Threshold={thr:.2f}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

Threshold=0.00: Precision=0.388, Recall=1.000, F1=0.559
Threshold=0.10: Precision=0.388, Recall=1.000, F1=0.559
Threshold=0.20: Precision=0.405, Recall=0.951, F1=0.568
Threshold=0.30: Precision=0.481, Recall=0.519, F1=0.499
Threshold=0.40: Precision=0.451, Recall=0.111, F1=0.179
Threshold=0.50: Precision=0.300, Recall=0.042, F1=0.073
Threshold=0.60: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.70: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.80: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.90: Precision=0.000, Recall=0.000, F1=0.000


## Cell 7: Random Forest (nonlinear diagnostics)

In [10]:
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

probs_rf = rf.predict_proba(X_test)[:,1]

for thr in np.arange(0, 1, 0.1):
    preds = (probs_logit > thr).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, preds, average='binary', zero_division=0
    )
    print(f"Threshold={thr:.2f}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

Threshold=0.00: Precision=0.388, Recall=1.000, F1=0.559
Threshold=0.10: Precision=0.388, Recall=1.000, F1=0.559
Threshold=0.20: Precision=0.405, Recall=0.951, F1=0.568
Threshold=0.30: Precision=0.481, Recall=0.519, F1=0.499
Threshold=0.40: Precision=0.451, Recall=0.111, F1=0.179
Threshold=0.50: Precision=0.300, Recall=0.042, F1=0.073
Threshold=0.60: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.70: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.80: Precision=0.000, Recall=0.000, F1=0.000
Threshold=0.90: Precision=0.000, Recall=0.000, F1=0.000


## Cell 8: Feature importance

In [9]:
pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

volatility     0.269523
spread_lag2    0.192028
spread_lag1    0.183882
zscore         0.179788
spread         0.174779
dtype: float64

## Final Conclusion

ML models exhibit conservative behavior across probability thresholds,
indicating weak predictability of fixed-horizon mean reversion.

This supports the interpretation of ML as a risk-filtering mechanism
rather than a deterministic signal generator.

### Interpretation of Probability Threshold Sweep

The threshold sweep reveals a clear precision–recall trade-off.
Lower thresholds capture most mean-reversion events but generate
substantial false positives, while higher thresholds suppress trading
activity entirely.

This behavior reflects the diffuse and regime-dependent nature of
mean reversion in financial spreads. Machine learning models therefore
serve as conservative risk filters rather than deterministic predictors.
